## MLflow's Model Registry

In [2]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///backend.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [4]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

[<Experiment: artifact_location='file:///e:/Documentos/Proyectos/Personal/mlops-zoomcamp-2024/02-experiment-tracking/mlruns/1', creation_time=1716832139112, experiment_id='1', last_update_time=1716832139112, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [5]:
client.create_experiment(name="my-cool-experiment")

'2'

Let's check the latest versions for the experiment with id `1`...

In [6]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [8]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 7ca469b037774e4fa9b08d531298453c, rmse: 6.3005
run id: a3d522e1d52d4966b56ab763152b4389, rmse: 6.3124
run id: 788fa3782a8748eb84676a17cf294f14, rmse: 6.3137
run id: d90f2e82dcea46768bc6511fe74d55b3, rmse: 6.3154
run id: 6bdca76522a74a2689e4b05cc55a8ff7, rmse: 6.3171


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [9]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [27]:
run_id = "f8bb3d6dbe3146fbbefb5cada0a01ea4"  # runs[3].info.run_id
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '2' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1717103814829, current_stage='None', description=None, last_updated_timestamp=1717103814829, name='nyc-taxi-regressor', run_id='f8bb3d6dbe3146fbbefb5cada0a01ea4', run_link=None, source='file:///e:/Documentos/Proyectos/Personal/mlops-zoomcamp-2024/02-experiment-tracking/mlruns/1/f8bb3d6dbe3146fbbefb5cada0a01ea4/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [28]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Staging
version: 2, stage: None


C:\Users\Almudena\AppData\Local\Temp\ipykernel_6328\669935608.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [15]:
model_version = 1
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

C:\Users\Almudena\AppData\Local\Temp\ipykernel_6328\1600074043.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1717102710164, current_stage='Staging', description=None, last_updated_timestamp=1717102986120, name='nyc-taxi-regressor', run_id='d90f2e82dcea46768bc6511fe74d55b3', run_link=None, source='file:///e:/Documentos/Proyectos/Personal/mlops-zoomcamp-2024/02-experiment-tracking/mlruns/1/d90f2e82dcea46768bc6511fe74d55b3/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [29]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1717102710164, current_stage='Staging', description='The model version 1 was transitioned to Staging on 2024-05-30', last_updated_timestamp=1717103822817, name='nyc-taxi-regressor', run_id='d90f2e82dcea46768bc6511fe74d55b3', run_link=None, source='file:///e:/Documentos/Proyectos/Personal/mlops-zoomcamp-2024/02-experiment-tracking/mlruns/1/d90f2e82dcea46768bc6511fe74d55b3/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [30]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [31]:
df = read_dataframe("../data/green_tripdata_2021-02.csv")

In [32]:
client.__dict__

{'_registry_uri': 'sqlite:///backend.db',
 '_tracking_client': <mlflow.tracking._tracking_service.client.TrackingServiceClient at 0x2957f59c4c0>,
 '_registry_client_lazy': <mlflow.tracking._model_registry.client.ModelRegistryClient at 0x2957fdac5e0>}

In [33]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'e:\\Documentos\\Proyectos\\Personal\\mlops-zoomcamp-2024\\02-experiment-tracking\\preprocessor'

In [34]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [35]:
X_test = preprocess(df, dv)

In [36]:
target = "duration"
y_test = df[target].values

In [43]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

OSError: No such file or directory: 'E:\Documentos\Proyectos\Personal\mlops-zoomcamp-2024\02-experiment-tracking\mlruns\1\d90f2e82dcea46768bc6511fe74d55b3\artifacts\model\.'

In [45]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

OSError: No such file or directory: 'E:\Documentos\Proyectos\Personal\mlops-zoomcamp-2024\02-experiment-tracking\mlruns\1\d90f2e82dcea46768bc6511fe74d55b3\artifacts\model\.'

In [40]:
client.transition_model_version_stage(
    name=model_name,
    version=2,
    stage="Production",
    archive_existing_versions=True
)

C:\Users\Almudena\AppData\Local\Temp\ipykernel_6328\1703833061.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1717103814829, current_stage='Production', description=None, last_updated_timestamp=1717103884335, name='nyc-taxi-regressor', run_id='f8bb3d6dbe3146fbbefb5cada0a01ea4', run_link=None, source='file:///e:/Documentos/Proyectos/Personal/mlops-zoomcamp-2024/02-experiment-tracking/mlruns/1/f8bb3d6dbe3146fbbefb5cada0a01ea4/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>